# Cipher Tools Library
A collection of tools developed along the way to help with deciphering

In [ ]:
import string

# A simple translation method.
# text: The text to translate, in upper case
# key: A substitution alphabet, usually in lower case so that the translated characters show up
def decode(text, key):
    table = str.maketrans(string.ascii_uppercase, key)
    print(text.upper().translate(table))

In [ ]:
import string

# Alternative version of the above, but this time preserving the case
def decode2(text, key):
    table = str.maketrans(string.ascii_letters, key.lower()+key.upper())
    print(text.translate(table))

In [ ]:
# A method to decode/encode a Vigenere cipher.  Case and punctuation are preserved.
# text: The text to encode
# key: The key to use
# encode: If set to True, will encode the given text instead of decoding it
def vigenere(text, key, encode=False):
    shifts = [ord(letter.upper()) - ord("A") for letter in key]
    key_length = len(key)
    key_index = 0
    for letter in text:
        letter_val = ord(letter)
        if letter.isupper():
            offset = ord("A")
        elif letter.islower():
            offset = ord("a")
        else:
            print(letter, end="")
            continue

        if encode:
            shift = shifts[key_index]
        else:
            shift = -shifts[key_index]

        print(chr((((letter_val - offset) + shift) % 26) + offset), end="")
        key_index = (key_index + 1) % key_length
    print()

In [ ]:
from collections import Counter

# Takes a string or list of items and counts the frequencies of those items
# data: The list or string to analyse
# max_values: The maximum number of values to display (set to None for no limit)
# no_columns: The amount of columns to use in the output
def frequency_analysis(data, max_values=30, no_columns=5):
    frequencies = Counter()
    for item in data:
        frequencies[item] += 1
    
    total = sum(frequencies.values())
    column = 1
    for item, frequency in frequencies.most_common(max_values):
        print(f"{item}: {frequency:2} ({frequency / total:.2%})", end=" " if column % no_columns else "\n")
        column += 1
    print("\n-----")

In [ ]:
import string

# Attempts to determine the keyword used for a substition cipher
# alphabet: The substitution alphabet
# TODO Separate out keyword from remainder of alphabet (find a stopping case!)
def decode_key(alphabet):
    for letter in string.ascii_lowercase:
        print(chr(alphabet.find(letter) + ord("a")), end="")
    print()

# Creates a substitution alphabet from a given keyword
def encode_key(keyword):
    key = list("." * 26)
    letters = list(string.ascii_lowercase)
    position = 0
    for letter in keyword:
        position = ord(letter)-ord("a")
        key[position] = letters.pop(0)
    index = (position + 1) % 26
    while index != position:
        if key[index] == ".":
            key[index] = letters.pop(0)
        index = (index + 1) % 26

    print("".join(key))


In [ ]:
# Tools to help decode Vigenere ciphers


# A function that searches for multiple ocurrences of letters of the specified size and prints out the most common ones
def pattern_finder(text, size):
    patterns = Counter()
    for index in range(len(text)):
        patterns[text[index:index+size]] += 1

    column = 0
    for item, pattern in patterns.most_common(30):
        print(f"{item}: {pattern:2}", end=" ")
        column += 1
        if column % 5 == 0:
            print()


# A function that searches the given text for a word and figures out where it would line up with a key of a given size
def get_alignment(text, word, key_size):
    print(word, end=" ")
    alignments = []
    position = -1
    while True:
        position = text.find(word, position+1)
        if position >= 0:
            alignment = position % key_size
            print(f"{position} ({alignment})", end=" ")
            alignments.append(alignment)
        else:
            break
    if alignments.count(alignments[0]) == len(alignments):
        print("All the same!", end="")
    print()


# A function to reverse search for a key that would convert the given word to an encoded word
def lookup_key(word, encoded_word):
    key = ""
    for letter, encoded_letter in zip(word, encoded_word):
        diff = (ord(encoded_letter) - ord(letter)) % 26
        key += chr(diff + ord('A'))
    print(key)

In [ ]:
def columnar_transposition(text, key):
    text = text.replace(" ", "")
    columns = dict()
    column_length = len(text) // len(key)
    count = 0
    for column in sorted(key):
        start_pos = count*column_length
        columns[column] = text[start_pos:start_pos+column_length]
        count += 1
    for row in range(column_length):
        for char in key:
            print(columns[char][row], end="")
